In [40]:
from pathlib import Path
from io import BytesIO
from PIL import Image
import numpy as np
import os, json, ast

root_path   = Path('../steam_data/')
games_path  = Path(root_path, 'steam_games.json')

output_path = Path(root_path, 'media')
if not os.path.isdir(output_path):
    os.mkdir(output_path)

data = []
with open(games_path) as fp:
    lines = fp.read().split('\n')
    for line in lines[:-1]:
        data.append(ast.literal_eval(line))

In [2]:
data[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [10]:
import requests

api_base_url = "http://store.steampowered.com/api/appdetails/"

gameid = 730
endpoint = f'{api_base_url}?appids={gameid}'

print(endpoint)
r = requests.get(endpoint)
print(r)

http://store.steampowered.com/api/appdetails/?appids=730
<Response [200]>


In [22]:
r.json()['730']['data'].keys()

dict_keys(['type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support', 'dlc', 'detailed_description', 'about_the_game', 'short_description', 'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements', 'linux_requirements', 'developers', 'publishers', 'packages', 'package_groups', 'platforms', 'metacritic', 'categories', 'genres', 'screenshots', 'movies', 'recommendations', 'achievements', 'release_date', 'support_info', 'background', 'content_descriptors'])

In [31]:
content = r.json()[str(gameid)]['data']
content

{'type': 'game',
 'name': 'Counter-Strike: Global Offensive',
 'steam_appid': 730,
 'required_age': 0,
 'is_free': True,
 'controller_support': 'full',
 'dlc': [1490530],
 'detailed_description': '<h1>Nova atualização</h1><p><a href="https://store.steampowered.com/app/1490530/CounterStrike_Global_Offensive__Operation_Broken_Fang/" target="_blank" rel="noreferrer"  id="dynamiclink_0" >https://store.steampowered.com/app/1490530/CounterStrike_Global_Offensive__Operation_Broken_Fang/</a></p><br><h1>Sobre o jogo</h1>Counter-Strike: Global Offensive (CS: GO) expandirá na jogabilidade de ação baseada em equipes na qual foi pioneiro quando foi lançado há 12 anos.<br>CS: GO contém novos mapas, personagens e armas, além de conter versões atualizadas de conteúdos do CS clássico (como de_dust2). Além disso, CS: GO introduzirá novos modos de jogo, criação de partidas (matchmaking), placares de líderes e mais.<br>&quot;Counter-Strike surpreendeu a indústria de jogos quando o improvável MOD virou o j

In [46]:
def download_media(content, gameid):
    header_image = content['header_image']  
    screenshots = content['screenshots']
    movies = content['movies']
    background = content['background']

    game_output_path = Path(output_path, str(gameid))
    if not os.path.isdir(game_output_path):
        os.mkdir(game_output_path)
    else:
        print(f'Game id {gameid} processed twice!')

    
    #### HEADER IMAGE
    try:
        url = content['header_image']
        response = requests.get(url)
        im = Image.open(BytesIO(response.content))    

        with open(Path(game_output_path, media+'.jpg'), 'w') as fp:
            im.save(fp)
    except:
        print(f'\ncould not process media {content[media]} from game {gameid}')
        pass

    #### SCREENSHOTS
    for k, screenshot in enumerate(content['screenshots']):

        try:
            url = screenshot['path_full']
            response = requests.get(url)
            im = Image.open(BytesIO(response.content)) 

            with open(Path(game_output_path, f'screenshot_{k}.jpg'), 'w') as fp:
                im.save(fp)

        except:
            print(f'\ncould not process screenshot {url} from game {gameid}')
            pass
        
    #### MOVIES
    for k, movie in enumerate(content['movies']):
        if k > 2: break

        if 'mp4' in movie.keys():
            url = movie['mp4']['480']
            save_name = f'movie_{k}_trailer.mp4' if 'trailer' in movie['name'].lower() else f'movie_{k}.mp4'
        else:
            url = movie['webm']['480']
            save_name = f'movie_{k}_trailer.webm' if 'trailer' in movie['name'].lower() else f'movie_{k}.webm'

        try:
            stream = requests.get(url, stream = True) 

            with open(Path(game_output_path, save_name), 'wb') as f: 
                for chunk in stream.iter_content(chunk_size = 1024*1024): 
                    if chunk: 
                        f.write(chunk)
        except:
            print(f'\ncould not process movie {url} from game {gameid}')
            pass
    

In [47]:
for i, game in enumerate(data):
    if i > 2: break
    
    print('\r{}/{} - {}: {} ######'.format(i, len(data), game['id'], game['app_name']), end='', flush=True)
    gameid = game['id']
    endpoint = f'{api_base_url}?appids={gameid}'

    r = requests.get(endpoint)
    content = r.json()[str(gameid)]['data']
    download_media(content, gameid)

######2/32135 - 670290: Real Pool 3D - Poolians ######